In [6]:
from collections import OrderedDict
import re
import os

import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import torch
from tqdm.notebook import tqdm
import ipywidgets
from plot_utils import get_model_names_for_degree

from eval import get_run_metrics, read_run_dir, get_model_from_run
from plot_utils import basic_plot, collect_results, relevant_model_names

%matplotlib inline
%load_ext autoreload
%autoreload 2
sns.set_theme('notebook', 'darkgrid')
palette = sns.color_palette('colorblind')

run_dir = "../models"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df = read_run_dir(run_dir)
df  # list all the runs in our run_dir

['chebyshev_linear_regression_big_2x' 'chebyshev_linear_regression_toy'
 'chebyshev_linear_regression_toy_0.2_noise'
 'chebyshev_linear_regression_toy_0.5_noise_0.00005_lr'
 'chebyshev_linear_regression_toy_11_take_1'
 'chebyshev_linear_regression_toy_16_layers'
 'chebyshev_linear_regression_toy_chebyshev_as_baseline_for_loss'
 'chebyshev_linear_regression_toy_just_base_8'
 'chebyshev_linear_regression_toy_noise'
 'chebyshev_linear_regression_toy_standard_0.00005_lr_no_noise_no_curriculum'
 'chebyshev_linear_regression_toy_uniform_sampling'
 'chebyshev_linear_regression_toy_weighted_mse' 'curriculum'
 'kernel_linear_regression_toy_basis_4'
 'kernel_linear_regression_toy_basis_4_dim_1'
 'kernel_linear_regression_toy_basis_4_normalized'
 'kernel_linear_regression_toy_basis_4_normalized_x_scale_2'
 'kernel_linear_regression_toy_dim_20_basis_2'
 'kernel_linear_regression_toy_just_basis_4_log' 'lr00005'
 'with_curriculum']
                                    run_id                          

,run_id,task,model,kwargs,num_tasks,num_examples,n_dims,n_layer,n_head,run_name
137,chebyshev-2x-0.0001,kernel_linear_regression,Transformer-plus,basis_dim=11_different_degrees=True_highest_de...,-1,-1,1,24,16,chebyshev_linear_regression_big_2x
0,c615f0ae-9309-49eb-a6cc-0c2680555c79,soft_prompting,chebyshev_linear_regression_toy,basis_dim=3_different_degrees=False_highest_de...,-1,-1,1,6,4,chebyshev_linear_regression_toy
108,a576aa50-f207-436f-9fe8-1d435e841041,soft_prompting_comparisions-copy,chebyshev_linear_regression_toy,basis_dim=5_fixed_coeffs=1_highest_degree=5_lo...,-1,-1,1,6,4,chebyshev_linear_regression_toy
109,eeaa5a62-debc-4b0e-955b-eeacc8cf2560,soft_prompting_comparisions-copy,chebyshev_linear_regression_toy,basis_dim=5_degree=5_perturbation=0.4,-1,-1,1,6,4,chebyshev_linear_regression_toy
110,50162e7c-8281-4e71-98a6-040b0eaa58ba,soft_prompting_comparisions-copy,chebyshev_linear_regression_toy,basis_dim=5_fixed_coeffs=0_highest_degree=5_lo...,-1,-1,1,6,4,chebyshev_linear_regression_toy
...,...,...,...,...,...,...,...,...,...,...
154,basis-4-normalized-x-scale-2,kernel_linear_regression,Transformer,basis_dim=4,-1,-1,5,12,8,kernel_linear_regression_toy_basis_4_normalize...
118,high-dimension-joint-linear-quadratic,kernel_linear_regression,Transformer,basis_dim=2,-1,-1,20,12,8,kernel_linear_regression_toy_dim_20_basis_2
136,a6c28d4b-4687-4631-9f46-1f5acbe1c18c,kernel_linear_regression,Transformer,basis_dim=4,-1,-1,5,12,8,kernel_linear_regression_toy_just_basis_4_log
161,lr_00005,kernel_linear_regression,Transformer,basis_dim=11_different_degrees=True_highest_de...,-1,-1,1,12,8,lr00005


In [9]:
task = "kernel_linear_regression"
#task = "sparse_linear_regression"
#task = "decision_tree"
#task = "relu_2nn_regression"
for run_id in os.listdir(os.path.join(run_dir, "soft_prompting_comparisons_long_training")):
    run_path = os.path.join(run_dir, "soft_prompting_comparisons_long_training", run_id)


    run_id_big = "small-1-11-5M-steps"
    run_path_big = os.path.join(run_dir, task, run_id_big)

    metrics = get_run_metrics(run_path, run_path_big, None, include_noise=False, ground_truth_loss=False, smoothing=0.00)  


torch.Size([1, 128])
PROMPT:  tensor([[[-0.6533],
         [ 1.1749],
         [-0.9855],
         [-0.0954]]], grad_fn=<UnsafeViewBackward0>)
Orthogonal component:  tensor(13.9499, grad_fn=<CopyBackwards>)
total norm:  tensor(24.8792, grad_fn=<CopyBackwards>)
../models/soft_prompting_comparisons_long_training/1830d38f-77d1-40eb-a6b4-d25f5184152c/metrics.json
no metrics found
0.2
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
torch.Size([64, 31, 5])
(64, 4)
t

# Plot pre-computed metrics

In [12]:
import numpy as np
def valid_row(r, run_id):
    return r.task == "soft_prompting_comparisons_long_training" and r.run_id == run_id

loss_performance = {}
def calculate_dist_mutual_info(task, task_args):
    from tasks import get_task_sampler, ChebyshevKernelLinearRegression, PolynomialSharedRoots
    from samplers import get_data_sampler, UniformSampler
    from npeet import entropy_estimators as ee
    data_sampler = UniformSampler(1)
    data =  data_sampler.sample_xs(20, 10000, 1)
    if task == "polynomial_shared_roots":
        task = PolynomialSharedRoots(n_dims=1, batch_size=10000, **task_args)
    else:
        task = ChebyshevKernelLinearRegression(n_dims=1, batch_size=10000, **task_args)
    ys = task.evaluate(data).squeeze()
    ys = ys.reshape(-1)
    data = data.reshape(-1)
    data = data.numpy()
    ys = ys.numpy()
    return ee.mi(data, ys), ee.entropy(ys[:, None])

losses = {}
mi = {}
en = {}

for run_id in os.listdir(os.path.join(run_dir, "soft_prompting_comparisons_long_training")):
    run_path = os.path.join(run_dir, "soft_prompting_comparisons_long_training", run_id)
    metrics = collect_results(run_dir, df, valid_row=lambda x: valid_row(x, run_id), smoothing=0.00)
    print(metrics.keys())
    print(metrics["degree-5"].keys())
    print(metrics["degree-5"]["chebyshev_linear_regression_toy Soft Prompt"]["mean"])
    soft_prompt_metrics = metrics["degree-5"]["chebyshev_linear_regression_toy Soft Prompt"]["mean"]
    baseline_metrics = metrics["degree-5"]["chebyshev_linear_regression_toy"]["mean"] 
    #print(np.array(soft_prompt_metrics)/np.array(baseline_metrics))
    #print(soft_prompt_metrics)
    #print(baseline_metrics)
    _, conf = get_model_from_run(run_path, only_conf=True)
    n_dims = conf.model.n_dims
    ylims = [2, 2, 0.2, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
    for degree in range(5, 6):
        models = get_model_names_for_degree(degree)
    #   print(models)
    #   print(metrics.keys())
    #   print(metrics["degree-" + str(degree)].keys())
        basic_plot(metrics["degree-" + str(degree)], models=models, ylim=ylims[degree-1])
        plt.title("Loss for polynomial degree " + str(degree))
        plt.savefig("graphs/degree-" + str(degree) + ".png", dpi=300, bbox_inches='tight')
        plt.show()
    #print(np.sum(np.array(soft_prompt_metrics))/np.sum(np.array(baseline_metrics)))
    task = conf.training.task
    key_args = conf.training.task_kwargs
    scale = -1
    print(task)
    if task != "polynomial_shared_roots":
        print(key_args)
        scale = key_args["scale"]

    if (task, scale) not in losses:
        losses[(task, scale)] = []
        mi[(task, scale)] = []
        en[(task, scale)] = []

    losses[(task, scale)].append(np.sum(np.array(soft_prompt_metrics))/np.sum(np.array(baseline_metrics)))
    m, e = calculate_dist_mutual_info(task, key_args)
    mi[(task, scale)].append(m)
    en[(task, scale)].append(e)

for (task, scale) in losses:
    plt.scatter(mi[(task, scale)], losses[(task, scale)])
    plt.xlabel("Mutual Information")
    plt.ylabel("Loss Ratio")
    plt.title("Mutual Information vs Loss Ratio for {}".format((task, scale)))
    plt.show()
    plt.close()

for (task, scale) in losses:
    plt.scatter(en[(task, scale)], losses[(task, scale)])
    plt.xlabel("Entropy of Y")
    plt.ylabel("Loss Ratio")
    plt.title("Mutual Information vs Loss Ratio for {}".format((task, scale)))
    plt.show()
    plt.close()


#metrics_smooth = collect_results(run_dir, df, valid_row=lambda x: valid_row(x, run_id), smoothing=0.01)
#metrics_big = collect_results(run_dir, df, valid_row= lambda x: valid_row(x, run_id_big), smoothing=0.00)


#for k, v in metrics_big.items():
#    for k2, v2 in v.items():
#        if "Transformer" in k2:
#            metrics[k][k2] = v2
#for k, v in metrics_smooth.items():
#    for k2, v2 in v.items():
#        if "Transformer" in k2:
#            metrics[k][k2+ " Smoothed"] = v2



dict_keys([])


KeyError: 'degree-5'

ValueError: too many values to unpack (expected 2)